In [1]:
# https://www.kaggle.com/carlosknows/taxis-in-mexico/code
from pyspark import SparkContext
from numpy import array
from math import sqrt
from pyspark.mllib.clustering import KMeans, KMeansModel

sc = SparkContext.getOrCreate()

In [2]:
df = sc.textFile("data/taxi/mex_clean.csv")
parsedData = df.map(lambda line: array([float(x) for x in line.split(',')]))

In [3]:
df.count()

12694

In [4]:
clusters = KMeans.train(parsedData, 7, maxIterations=100000, initializationMode="random")

In [5]:
clusters.clusterCenters

[array([3.89548612e+06, 1.55053750e+04, 2.14581250e+03]),
 array([2.68000000e+02, 2.30700000e+03, 3.67113629e+08]),
 array([1058915.73493976,   19227.03614458,    2327.61445783]),
 array([9927.55946612, 6569.45380154, 1321.54905855]),
 array([9.92000000e+02, 3.72766667e+03, 1.34058766e+09]),
 array([1.94200000e+03, 6.68633333e+03, 3.44512221e+09]),
 array([16570949.,    56490.,    51500.])]

In [6]:
# Evaluate clustering by computing Within Set Sum of Squared Errors
def error(point):
    center = clusters.centers[clusters.predict(point)]
    return sqrt(sum([x**2 for x in (point - center)]))

In [7]:
WSSSE = parsedData.map(lambda point: error(point)).reduce(lambda x, y: x + y)
print("Within Set Sum of Squared Error = " + str(WSSSE))

Within Set Sum of Squared Error = 2822042810.485467


In [8]:
# Save and load model
clusters.save(sc, "data/taxi/KMeansModel")
sameModel = KMeansModel.load(sc, "data/taxi/KMeansModel")
# $example off$

sc.stop()